In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt

# Correct path to where patient folders are stored
data_path = r"C:/Users/N.Kaushalee/Desktop/Final year project/Data sets/archive"

# Example patient
patient_id = "BraTS2021_00495"
patient_path = os.path.join(data_path, patient_id)

# Path to files inside the patient's folder
flair_path = os.path.join(patient_path, patient_id + "_flair.nii.gz")
t1_path = os.path.join(patient_path, patient_id + "_t1.nii.gz")
t1ce_path = os.path.join(patient_path, patient_id + "_t1ce.nii.gz")
t2_path = os.path.join(patient_path, patient_id + "_t2.nii.gz")
seg_path = os.path.join(patient_path, patient_id + "_seg.nii.gz")

# Load and display
flair = nib.load(flair_path).get_fdata()
seg = nib.load(seg_path).get_fdata()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.imshow(flair[:, :, 80], cmap="gray")
plt.title("FLAIR")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(seg[:, :, 80], cmap="gray")
plt.title("Segmentation")
plt.axis("off")

plt.show()


In [3]:
pip install cv2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
ERROR: No matching distribution found for cv2
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import numpy as np
import nibabel as nib
import cv2


In [2]:

# Define preprocessing function
def load_and_preprocess(patient_path, patient_id, image_size=(128, 128)):
    # List of MRI modalities
    modalities = ['flair', 't1', 't1ce', 't2']
    
    # List to store all modality images
    images = []
    
    # Loop over each modality to load the MRI images
    for modality in modalities:
        # Construct file path for each modality (e.g., flair, t1, t1ce, t2)
        file_path = os.path.join(patient_path, f"{patient_id}_{modality}.nii.gz")
        
        # Load the MRI image using nibabel
        img = nib.load(file_path).get_fdata()
        
        # Normalize the image: Scale the pixel values to range 0-1
        img = (img - img.min()) / (img.max() - img.min())  # Normalize the image
        
        # Optionally rotate (if needed) and resize the image
        img = np.rot90(img, k=1, axes=(0, 1))  # Rotate if images are not aligned correctly
        images.append(img)  # Append the image to the list of images
    
    # Stack all 4 modalities (FLAIR, T1, T1CE, T2) into one 4-channel image
    images = np.stack(images, axis=-1)  # Shape: (H, W, Slices, 4)
    images = np.transpose(images, (2, 0, 1, 3))  # Shape: (Slices, H, W, 4)

    # Load the segmentation mask
    seg_path = os.path.join(patient_path, f"{patient_id}_seg.nii.gz")
    mask = nib.load(seg_path).get_fdata()
    
    # Normalize and rotate the mask (segmentation is binary)
    mask = np.rot90(mask, k=1, axes=(0, 1))
    mask = np.transpose(mask, (2, 0, 1))  # Shape: (Slices, H, W)

    # Resize images and masks to the target size (e.g., 128x128)
    X = []  # Input data (images)
    Y = []  # Output data (masks)
    
    for i in range(images.shape[0]):  # Iterate over all slices
        # Resize each slice of the images (input MRI modalities)
        img_resized = cv2.resize(images[i], image_size, interpolation=cv2.INTER_LINEAR)
        
        # Resize the corresponding mask (segmentation)
        mask_resized = cv2.resize(mask[i], image_size, interpolation=cv2.INTER_NEAREST)
        
        X.append(img_resized)  # Append the resized input image
        Y.append(mask_resized)  # Append the resized mask
    
    X = np.array(X)  # Shape: (num_slices, 128, 128, 4)
    Y = np.array(Y)  # Shape: (num_slices, 128, 128)

    return X, Y

# Example usage with your dataset path
data_path = "archive/BraTS2021_Training_Data"
# patient_id = "BraTS2021_00495"  # Change this if you want to process another patient
# patient_path = os.path.join(data_path, patient_id)

# # Preprocess the data for the given patient
# X, Y = load_and_preprocess(patient_path, patient_id)

results = []

for folder_name in os.listdir(data_path):
    patient_path = os.path.join(data_path, folder_name)

    if os.path.isdir(patient_path):
        patient_id = folder_name
        print(folder_name)
        X, Y = load_and_preprocess(patient_path, patient_id)
        results.append((patient_id, X, Y))


# Check the shapes of the preprocessed data
print("Input shape:", X.shape)  # (num_slices, 128, 128, 4)
print("Mask shape:", Y.shape)   # (num_slices, 128, 128)


BraTS2021_00000
BraTS2021_00002
BraTS2021_00003
BraTS2021_00005
BraTS2021_00006
BraTS2021_00008
BraTS2021_00009
BraTS2021_00011
BraTS2021_00012
BraTS2021_00014
BraTS2021_00016
BraTS2021_00017
BraTS2021_00018
BraTS2021_00019
BraTS2021_00020
BraTS2021_00021
BraTS2021_00022
BraTS2021_00024
BraTS2021_00025
BraTS2021_00026
BraTS2021_00028
BraTS2021_00030
BraTS2021_00031
BraTS2021_00032
BraTS2021_00033
BraTS2021_00035
BraTS2021_00036
BraTS2021_00043
BraTS2021_00044
BraTS2021_00045
BraTS2021_00046
BraTS2021_00048
BraTS2021_00049
BraTS2021_00051
BraTS2021_00052
BraTS2021_00053
BraTS2021_00054
BraTS2021_00056
BraTS2021_00058
BraTS2021_00059
BraTS2021_00060
BraTS2021_00061
BraTS2021_00062
BraTS2021_00063
BraTS2021_00064
BraTS2021_00066
BraTS2021_00068
BraTS2021_00070
BraTS2021_00071
BraTS2021_00072
BraTS2021_00074
BraTS2021_00077
BraTS2021_00078
BraTS2021_00081
BraTS2021_00084
BraTS2021_00085
BraTS2021_00087
BraTS2021_00088
BraTS2021_00089
BraTS2021_00090
BraTS2021_00094
BraTS2021_00095
BraTS202

: 

In [ ]:
import numpy as np
from sklearn.utils import shuffle

# Shuffle the patients (for randomness)
results = shuffle(results, random_state=42)

# Define how many patients for each split
num_train = 70
num_val = 20
num_test = 10

# Split by number of patients
train_patients = results[:num_train]
val_patients = results[num_train:num_train + num_val]
test_patients = results[num_train + num_val:num_train + num_val + num_test]

# Helper function to flatten the data
def extract_X_Y(patients):
    X = np.concatenate([x for _, x, _ in patients], axis=0)
    Y = np.concatenate([y for _, _, y in patients], axis=0)
    return X, Y

# Prepare datasets
X_train, Y_train = extract_X_Y(train_patients)
X_val, Y_val     = extract_X_Y(val_patients)
X_test, Y_test   = extract_X_Y(test_patients)

# Check shapes
print("Train X:", X_train.shape, "Y:", Y_train.shape)
print("Val X:  ", X_val.shape,   "Y:", Y_val.shape)
print("Test X: ", X_test.shape,  "Y:", Y_test.shape)



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_unet(input_size=(128, 128, 4)):
    inputs = layers.Input(input_size)

    # Contracting path (Encoder)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D((2, 2))(conv3)

    # Bottleneck
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    # Expansive path (Decoder)
    up1 = layers.UpSampling2D((2, 2))(conv4)
    up1 = layers.concatenate([up1, conv3], axis=-1)
    conv5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
    conv5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

    up2 = layers.UpSampling2D((2, 2))(conv5)
    up2 = layers.concatenate([up2, conv2], axis=-1)
    conv6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    conv6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

    up3 = layers.UpSampling2D((2, 2))(conv6)
    up3 = layers.concatenate([up3, conv1], axis=-1)
    conv7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
    conv7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

    # Output layer
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv7)

    # Build the model
    model = models.Model(inputs, outputs)
    return model

# Build U-Net model
model = build_unet(input_size=(128, 128, 4))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summarize the model architecture
model.summary()


In [ ]:
# Train the model using explicit validation data
history = model.fit(
    X_train, Y_train,
    epochs=10,
    batch_size=4,
    validation_data=(X_val, Y_val)  # <-- Use explicit validation set
)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"Test accuracy: {test_acc:.4f}")



In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
model.save('brain_tumor_segmentation_model.h5')

In [ ]:
predictions = model.predict(X_test)
print(predictions.shape)  # (num_test_samples, 128, 128, 1)


In [ ]:
import matplotlib.pyplot as plt

# Visualize the first test image, its true mask, and predicted mask
plt.figure(figsize=(12, 4))

# Input image
plt.subplot(1, 3, 1)
plt.imshow(X_test[0, :, :, 0], cmap='gray')  # First MRI modality (e.g., FLAIR)
plt.title("Input Image")
plt.axis("off")

# True mask
plt.subplot(1, 3, 2)
plt.imshow(Y_test[0], cmap='gray')
plt.title("True Mask")
plt.axis("off")

# Predicted mask
plt.subplot(1, 3, 3)
plt.imshow(predictions[0, :, :, 0], cmap='gray')
plt.title("Predicted Mask")
plt.axis("off")

plt.show()


In [ ]:
# Pick a random test image and its corresponding mask
index = np.random.randint(0, len(X_test))  # random index

# Get the test image and ground truth mask
test_image = X_test[index]
true_mask = Y_test[index]

# Predict the mask using the trained model
predicted_mask = model.predict(np.expand_dims(test_image, axis=0))[0]  # Predict
predicted_mask = (predicted_mask > 0.5).astype(np.uint8)  # Threshold

# Plot the input image, ground truth mask, and predicted mask
plt.figure(figsize=(15,5))

# Input Image
plt.subplot(1,3,1)
plt.imshow(test_image[:,:,0], cmap='gray')
plt.title('Input MRI Image')
plt.axis('off')

# Ground Truth Mask
plt.subplot(1, 3, 2)
plt.imshow(true_mask, cmap='gray')  # No third index here
plt.title('Ground Truth Mask')
plt.axis('off')

# Predicted Mask
plt.subplot(1,3,3)
plt.imshow(predicted_mask[:,:,0], cmap='gray')
plt.title('Predicted Mask')
plt.axis('off')

plt.show()


In [ ]:
print(f"Sum of ground truth mask: {np.sum(true_mask)}")
print(f"Sum of predicted mask: {np.sum(predicted_mask)}")


In [ ]:
import numpy as np

def dice_coefficient(y_true, y_pred):
    """
    Calculate the Dice similarity coefficient between two binary masks.
    
    Parameters:
    y_true (numpy array): Ground truth mask (binary)
    y_pred (numpy array): Predicted mask (binary)
    
    Returns:
    float: Dice similarity score
    """
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    
    intersection = np.sum(y_true_f * y_pred_f)
    dice_score = (2. * intersection) / (np.sum(y_true_f) + np.sum(y_pred_f))
    return dice_score


In [ ]:
# Assuming you have X_test (input images) and y_test (ground truth masks)
dice_scores = []

# Evaluate the model on the test set
for i in range(len(X_test)):
    test_image = X_test[i]
    true_mask = Y_test[i]
    
    # Make predictions with the model
    predicted_mask = model.predict(np.expand_dims(test_image, axis=0))[0]
    
    # Convert the predicted mask to binary (0 or 1)
    predicted_mask = (predicted_mask > 0.5).astype(np.uint8)

    # Compute the Dice similarity coefficient
    dice_score = dice_coefficient(true_mask, predicted_mask)
    dice_scores.append(dice_score)

# Calculate the average Dice score over the test set
average_dice_score = np.mean(dice_scores)
print(f"Average Dice Similarity Score: {average_dice_score}")


In [ ]:
model.summary()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.transform import rotate, resize

def augment_data(image, mask):
    # Apply random rotation
    angle = np.random.uniform(-15, 15)
    image = rotate(image, angle, mode='wrap')
    mask = rotate(mask, angle, mode='wrap')

    # Random zoom
    zoom_factor = np.random.uniform(0.8, 1.2)
    image = resize(image, (image.shape[0] * zoom_factor, image.shape[1] * zoom_factor, image.shape[2] * zoom_factor), mode='constant')
    mask = resize(mask, (mask.shape[0] * zoom_factor, mask.shape[1] * zoom_factor, mask.shape[2] * zoom_factor), mode='constant')

    return image, mask


In [ ]:
def preprocess_image(image, new_shape=(128, 128, 128)):
    # Normalize to 0-1
    image = image / np.max(image)
    # Resize the image to the desired shape
    image = resize(image, new_shape)
    return image


In [ ]:
from tensorflow.keras import layers, models

def unet_3d(input_size=(128, 128, 128, 1)):
    inputs = layers.Input(input_size)
    # Downsampling path (contracting path)
    conv1 = layers.Conv3D(32, 3, activation='relu', padding='same')(inputs)
    pool1 = layers.MaxPooling3D(2)(conv1)

    conv2 = layers.Conv3D(64, 3, activation='relu', padding='same')(pool1)
    pool2 = layers.MaxPooling3D(2)(conv2)

    # Bottom layer
    conv3 = layers.Conv3D(128, 3, activation='relu', padding='same')(pool2)

    # Upsampling path (expanding path)
    up1 = layers.UpSampling3D(2)(conv3)
    conv4 = layers.Conv3D(64, 3, activation='relu', padding='same')(up1)

    up2 = layers.UpSampling3D(2)(conv4)
    conv5 = layers.Conv3D(32, 3, activation='relu', padding='same')(up2)

    output = layers.Conv3D(1, 1, activation='sigmoid', padding='same')(conv5)

    model = models.Model(inputs, output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
model = unet_3d()
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=50, batch_size=1)


In [ ]:
def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))
